In [ ]:
from ortools.sat.python import cp_model
from datetime import datetime, timedelta

class Team:
    def __init__(self, name, stadium):
        self.name = name
        self.stadium = stadium

class Stadium:
    def __init__(self, name, unavailable_dates=None):
        self.name = name
        self.unavailable_dates = set()
        if unavailable_dates:
            for date in unavailable_dates:
                if isinstance(date, str):
                    dt = datetime.fromisoformat(date)
                    self.unavailable_dates.add(dt.date())
                elif isinstance(date, datetime):
                    self.unavailable_dates.add(date.date())
                else:
                    try:
                        self.unavailable_dates.add(date)
                    except:
                        pass

    def is_available_on(self, date):
        d = date.date() if isinstance(date, datetime) else date
        return d not in self.unavailable_dates

class Schedule:
    def __init__(self, teams, start_date, max_consecutive_away, max_total_breaks):
        self.teams = teams
        self.start_date = datetime.fromisoformat(start_date) if isinstance(start_date, str) else start_date
        self.max_consecutive_away = max_consecutive_away
        self.num_teams = len(teams)
        self.num_days = len(teams) - 1
        model = cp_model.CpModel()

        self.X = {}
        for i in range(self.num_teams):
            for j in range(self.num_teams):
                if i == j:
                    continue
                for d in range(self.num_days):
                    if not teams[i].stadium.is_available_on(self.start_date + timedelta(days=d)):
                        continue
                    self.X[(i, j, d)] = model.NewBoolVar(f"match_{i}_{j}_day{d+1}")

        # Chaque paire d'équipes joue au plus une fois
        for i in range(self.num_teams):
            for j in range(i + 1, self.num_teams):
                model.Add(
                    sum(self.X.get((i, j, d), 0) for d in range(self.num_days)) +
                    sum(self.X.get((j, i, d), 0) for d in range(self.num_days))
                    <= 1
                )

        """
        # Une équipe joue un seul match par jour
        for t in range(self.num_teams):
            for d in range(self.num_days):
                model.Add(
                    sum(self.X.get((t, o, d), 0) for o in range(self.num_teams) if t != o) +
                    sum(self.X.get((o, t, d), 0) for o in range(self.num_teams) if t != o)
                    <= 1
                )

        # Chaque équipe joue au moins un match
        for t in range(self.num_teams):
            model.Add(
                sum(self.X.get((t, o, d), 0) for o in range(self.num_teams) if t != o for d in range(self.num_days)) +
                sum(self.X.get((o, t, d), 0) for o in range(self.num_teams) if t != o for d in range(self.num_days))
                >= 1
            )
        """
        for i in range(self.num_teams):
            for j in range(i + 1, self.num_teams):
                model.Add(
                    sum(self.X.get((i, j, d), 0) for d in range(self.num_days)) +
                    sum(self.X.get((j, i, d), 0) for d in range(self.num_days))
                    <= 1
                )

        # Limite de breaks consécutifs à l'extérieur
        for t in range(self.num_teams):
            if self.max_consecutive_away < self.num_days:
                for d0 in range(self.num_days - self.max_consecutive_away):
                    model.Add(
                        sum(self.X.get((o, t, d), 0)
                            for d in range(d0, d0 + self.max_consecutive_away + 1)
                            for o in range(self.num_teams) if o != t)
                        <= self.max_consecutive_away
                    )

        # 🔥 Breaks
        self.B = {}
        self.total_breaks = {}

        for t in range(self.num_teams):
            for d in range(self.num_days - 1):
                b = model.NewBoolVar(f"break_t{t}_day{d}")
                self.B[(t, d)] = b

                # Est-ce que l'équipe joue à domicile ou extérieur au jour d ?
                is_home_d = model.NewBoolVar(f"is_home_day{d}_team{t}")
                is_away_d = model.NewBoolVar(f"is_away_day{d}_team{t}")
                model.Add(
                    sum(self.X.get((t, o, d), 0) for o in range(self.num_teams) if o != t) == 1
                ).OnlyEnforceIf(is_home_d)
                model.Add(
                    sum(self.X.get((o, t, d), 0) for o in range(self.num_teams) if o != t) == 1
                ).OnlyEnforceIf(is_away_d)
                model.AddBoolOr([is_home_d, is_away_d])  # il joue ce jour-là

                # Idem pour jour d+1
                is_home_d1 = model.NewBoolVar(f"is_home_day{d+1}_team{t}")
                is_away_d1 = model.NewBoolVar(f"is_away_day{d+1}_team{t}")
                model.Add(
                    sum(self.X.get((t, o, d + 1), 0) for o in range(self.num_teams) if o != t) == 1
                ).OnlyEnforceIf(is_home_d1)
                model.Add(
                    sum(self.X.get((o, t, d + 1), 0) for o in range(self.num_teams) if o != t) == 1
                ).OnlyEnforceIf(is_away_d1)
                model.AddBoolOr([is_home_d1, is_away_d1])  # il joue aussi ce jour-là

                # Break si deux fois à domicile ou deux fois à l'extérieur
                is_home_break = model.NewBoolVar(f"is_home_break_{t}_{d}")
                is_away_break = model.NewBoolVar(f"is_away_break_{t}_{d}")

                model.AddBoolAnd([is_home_d, is_home_d1]).OnlyEnforceIf(is_home_break)
                model.AddBoolOr([is_home_d.Not(), is_home_d1.Not()]).OnlyEnforceIf(is_home_break.Not())

                model.AddBoolAnd([is_away_d, is_away_d1]).OnlyEnforceIf(is_away_break)
                model.AddBoolOr([is_away_d.Not(), is_away_d1.Not()]).OnlyEnforceIf(is_away_break.Not())

                model.AddMaxEquality(b, [is_home_break, is_away_break])

            self.total_breaks[t] = model.NewIntVar(0, self.num_days, f"total_breaks_t{t}")
            model.Add(self.total_breaks[t] == sum(self.B[(t, d)] for d in range(self.num_days - 1)))

        # 💣 Total global de breaks
        global_breaks = model.NewIntVar(0, self.num_teams * (self.num_days - 1), "global_breaks")
        model.Add(global_breaks == sum(self.total_breaks[t] for t in range(self.num_teams)))
        model.Add(global_breaks == max_total_breaks)

        # Résolution
        solver = cp_model.CpSolver()
        solver.parameters.max_time_in_seconds = 1000
        result = solver.Solve(model)

        self.schedule = []
        if result in (cp_model.OPTIMAL, cp_model.FEASIBLE):
            for d in range(self.num_days):
                matches = []
                for i in range(self.num_teams):
                    for j in range(self.num_teams):
                        if i != j and (i, j, d) in self.X and solver.Value(self.X[(i, j, d)]) == 1:
                            matches.append((i, j))
                self.schedule.append(matches)
        else:
            self.schedule = None

    def print_schedule(self):
        if self.schedule is None:
            print("Aucun calendrier valide n'a été trouvé.")
        else:
            for day, matches in enumerate(self.schedule, start=1):
                date_str = (self.start_date + timedelta(days=day - 1)).strftime("%Y-%m-%d")
                print(f"Journée {day} ({date_str}) :")
                for (home_idx, away_idx) in matches:
                    home_team = self.teams[home_idx].name
                    away_team = self.teams[away_idx].name
                    print(f"  {home_team} (domicile) vs {away_team} (extérieur)")


In [45]:

# Exemple d'utilisation
if __name__ == "__main__":
    n = 10
    stadiums = [Stadium(f"Stade {i + 1}") for i in range(n)]
    teams = [Team(f"Équipe {chr(65 + i)}", stadiums[i]) for i in range(n)]

    schedule = Schedule(
        teams,
        start_date="2025-04-01",
        max_consecutive_away=2,
        max_total_breaks=n-2  # 💥 ICI tu choisis le nombre de breaks max autorisé
    )

    schedule.print_schedule()


Starting CP-SAT solver v9.12.4544
Parameters: max_time_in_seconds: 1000 log_search_progress: true num_search_workers: 4

Initial satisfaction model '': (model_fingerprint: 0xb2b6417e812a8abf)
#Variables: 1'371
  - 1'370 Booleans in [0,1]
  - 1 in [0,80]
#kBoolAnd: 160 (#enforced: 160) (#literals: 480)
#kLinMax: 80 (#expressions: 160)
#kLinear1: 1
#kLinearN: 536 (#terms: 9'221)

Starting presolve at 0.00s
  9.37e-04s  0.00e+00d  [DetectDominanceRelations] 
  1.15e-02s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  3.57e-05s  0.00e+00d  [ExtractEncodingFromLinear] #potential_supersets=455 
  4.55e-04s  0.00e+00d  [DetectDuplicateColumns] 
  4.70e-05s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 3'187 nodes and 11'780 arcs.
[Symmetry] Symmetry computation done. time: 0.0152443 dtime: 0.0249116
[Symmetry] #generators: 9, average support size: 562.222
[Symmetry] 33 orbits on 1370 variables with sizes: 180,180,180,180,90,20,20,20,20

12 equipe = 9m23sec

In [44]:
from ortools.sat.python import cp_model
from datetime import datetime, timedelta


class Team:
    def __init__(self, name, stadium):
        self.name = name
        self.stadium = stadium


class Stadium:
    def __init__(self, name, unavailable_dates=None):
        self.name = name
        self.unavailable_dates = set(
            datetime.fromisoformat(date).date() if isinstance(date, str) else date
            for date in (unavailable_dates or [])
        )

    def is_available_on(self, date):
        return date.date() not in self.unavailable_dates


class Schedule:
    def __init__(self, teams, start_date, max_consecutive_away, max_total_breaks):
        self.teams = teams
        self.start_date = datetime.fromisoformat(start_date) if isinstance(start_date, str) else start_date
        self.num_teams = len(teams)
        self.num_days = self.num_teams - 1
        self.max_consecutive_away = max_consecutive_away

        model = cp_model.CpModel()
        self.X = {}

        # Création des variables uniquement si le match est possible
        for i in range(self.num_teams):
            for j in range(self.num_teams):
                if i != j:
                    for d in range(self.num_days):
                        match_date = self.start_date + timedelta(days=d)
                        if teams[i].stadium.is_available_on(match_date):
                            self.X[(i, j, d)] = model.NewBoolVar(f"match_{i}_{j}_day{d+1}")

        # 🔹 Une paire d'équipes joue au maximum une fois
        for i in range(self.num_teams):
            for j in range(i + 1, self.num_teams):
                model.Add(
                    sum(self.X.get((i, j, d), 0) + self.X.get((j, i, d), 0) for d in range(self.num_days)) <= 1
                )

        # 🔹 Une équipe ne joue qu’un seul match par jour
        for t in range(self.num_teams):
            for d in range(self.num_days):
                model.Add(
                    sum(self.X.get((t, o, d), 0) + self.X.get((o, t, d), 0) for o in range(self.num_teams) if t != o) <= 1
                )

        # 🔹 Chaque équipe doit jouer au moins un match
        for t in range(self.num_teams):
            model.Add(
                sum(self.X.get((t, o, d), 0) + self.X.get((o, t, d), 0) for o in range(self.num_teams) if t != o for d in range(self.num_days)) >= 1
            )

        # 🔹 Limite des matchs consécutifs à l'extérieur
        for t in range(self.num_teams):
            for d0 in range(self.num_days - self.max_consecutive_away):
                model.Add(
                    sum(self.X.get((o, t, d), 0) for d in range(d0, d0 + self.max_consecutive_away + 1) for o in range(self.num_teams) if o != t) <= self.max_consecutive_away
                )

        # 🔥 Gestion des breaks (deux matchs consécutifs à domicile ou à l'extérieur)
        self.B = {}
        for t in range(self.num_teams):
            for d in range(self.num_days - 1):
                self.B[(t, d)] = model.NewBoolVar(f"break_t{t}_day{d}")

                is_home_d = model.NewBoolVar(f"is_home_t{t}_d{d}")
                is_home_d1 = model.NewBoolVar(f"is_home_t{t}_d{d+1}")
                is_away_d = model.NewBoolVar(f"is_away_t{t}_d{d}")
                is_away_d1 = model.NewBoolVar(f"is_away_t{t}_d{d+1}")

                model.Add(is_home_d == sum(self.X.get((t, o, d), 0) for o in range(self.num_teams) if o != t))
                model.Add(is_home_d1 == sum(self.X.get((t, o, d + 1), 0) for o in range(self.num_teams) if o != t))
                model.Add(is_away_d == sum(self.X.get((o, t, d), 0) for o in range(self.num_teams) if o != t))
                model.Add(is_away_d1 == sum(self.X.get((o, t, d + 1), 0) for o in range(self.num_teams) if o != t))

                both_home = model.NewBoolVar(f"both_home_t{t}_d{d}")
                both_away = model.NewBoolVar(f"both_away_t{t}_d{d}")

                model.AddBoolAnd([is_home_d, is_home_d1]).OnlyEnforceIf(both_home)
                model.AddBoolAnd([is_away_d, is_away_d1]).OnlyEnforceIf(both_away)

                model.AddMaxEquality(self.B[(t, d)], [both_home, both_away])

        # 🔹 Nombre total de breaks
        global_breaks = model.NewIntVar(0, self.num_teams * (self.num_days - 1), "global_breaks")
        model.Add(global_breaks == sum(self.B[t, d] for t in range(self.num_teams) for d in range(self.num_days - 1)))
        model.Add(global_breaks <= max_total_breaks)

        # 🚀 Résolution avec paramètres optimisés
        solver = cp_model.CpSolver()
        solver.parameters.num_search_workers = 4  # Parallélisation
        solver.parameters.log_search_progress = True
        solver.parameters.max_time_in_seconds = 1000

        result = solver.Solve(model)
        self.schedule = []

        if result in (cp_model.OPTIMAL, cp_model.FEASIBLE):
            for d in range(self.num_days):
                matches = [(i, j) for i in range(self.num_teams) for j in range(self.num_teams)
                           if i != j and (i, j, d) in self.X and solver.Value(self.X[(i, j, d)]) == 1]
                self.schedule.append(matches)
        else:
            self.schedule = None

    def print_schedule(self):
        if self.schedule is None:
            print("Aucun calendrier valide trouvé.")
        else:
            for day, matches in enumerate(self.schedule, start=1):
                date_str = (self.start_date + timedelta(days=day - 1)).strftime("%Y-%m-%d")
                print(f"Journée {day} ({date_str}) :")
                for home_idx, away_idx in matches:
                    print(f"  {self.teams[home_idx].name} (domicile) vs {self.teams[away_idx].name} (extérieur)")
